**TM10007 Machine Learning**

## Import dependencies and data

In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np
import shutil
import subprocess
import shutil
import os
import stat
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from os import path
from sklearn import model_selection
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, fbeta_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.utils import resample
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

## if data.csv is not present download it from github
if not os.path.isfile("ecg_data.csv"):
    ## clone repo from githun
    if not os.path.isdir("tm10007_ml"):
        !git clone https://github.com/jveenland/tm10007_ml.git
    ## extract zip file
    if not os.path.isfile("tm10007_ml/ecg/ecg_data.csv"):
        with zipfile.ZipFile('tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
            zip_ref.extractall('tm10007_ml/ecg')
    ## move data file to root folder
    shutil.move('tm10007_ml/ecg/ecg_data.csv', 'ecg_data.csv')

    ## Delete cloned repo
    for root, dirs, files in os.walk("./tm10007_ml"):  
        for dir in dirs:
            os.chmod(path.join(root, dir), stat.S_IRWXU)
        for file in files:
            os.chmod(path.join(root, file), stat.S_IRWXU)
    shutil.rmtree('./tm10007_ml')

data = pd.read_csv('ecg_data.csv', index_col=0)

## Inspect imported data and clean missing data

### Plot number of missing data per features and per patient

In [ ]:
# exclude label column
values_features = data.drop(['label'], axis=1)

# plot the distribution of missing data per feature
num_zeros_features = (values_features == 0).sum(axis=0)
plt.scatter(range(len(num_zeros_features)),num_zeros_features)
plt.title("Distribution of number of missing data per feature")
plt.ylabel('Number of missing data')
plt.xlabel('Feature')
plt.show()

# plot the distribution of missing data per patient
num_zeros_patients=(values_features == 0).sum(axis=1)
plt.scatter(range(len(num_zeros_patients)),num_zeros_patients)
plt.title("Distribution of number of missing data per patient")
plt.ylabel('Number of missing data')
plt.xlabel('Patient')
plt.show()

### Delete rows with missing data

In [ ]:
values_data = data.drop(['label'], axis=1)
mask = (values_data != 0).all(axis=1)
clean_data=data[mask]
dirty_data=data[~mask]

### Plot number of missing data per features and per patient after data cleanup

In [ ]:
# exclude label column
values_clean_data = clean_data.drop(['label'], axis=1)
# plot the distribution of missing data per feature
num_zeros_features = (values_clean_data == 0).sum(axis=0)
plt.scatter(range(len(num_zeros_features)),num_zeros_features)
plt.title("Distribution of number of missing data per feature")
plt.ylabel('Number of missing data')
plt.xlabel('Feature')
plt.show()

# plot the distribution of missing data per patient
num_zeros_patients=(values_clean_data == 0).sum(axis=1)
plt.scatter(range(len(num_zeros_patients)),num_zeros_patients)
plt.title("Distribution of number of missing data per patient")
plt.ylabel('Number of missing data')
plt.xlabel('Patient')
plt.show()

## Split data into test and train data

In [ ]:
def clean_outlier(dataframe):
  new_dfs = []
  for i in range(9001):
        data=dataframe[dataframe.columns[i]]
        Q1 = np.percentile(data, 25)
        Q3 = np.percentile(data, 75)
        IQD = Q3 - Q1
        Upper_outlier_boundary = Q3 + (1.5 * IQD)
        Lower_outlier_boundary = Q1 - (1.5 * IQD)
        clipped_col = pd.DataFrame({dataframe.columns[i]: dataframe[dataframe.columns[i]].clip(Lower_outlier_boundary, Upper_outlier_boundary)})
        new_dfs.append(clipped_col)
  clean_outlier_data = pd.concat(new_dfs, axis=1)
  return clean_outlier_data


In [ ]:
ECG_ABNORMAL = clean_data[clean_data['label'] == 1]
ECG_NORMAL = clean_data[clean_data['label'] == 0]

x_train_abnormal, x_test_abnormal = model_selection.train_test_split(ECG_ABNORMAL, test_size=0.2, random_state=30)
x_train_normal, x_test_normal = model_selection.train_test_split(ECG_NORMAL,test_size = 0.2,random_state=30)

train_set = pd.concat([x_train_normal,x_train_abnormal])
test_set_new = pd.concat([x_test_normal,x_test_abnormal])

ECG_TRAIN_ABNORMAL = train_set[train_set['label'] == 1]
ECG_TRAIN_NORMAL = train_set[train_set['label'] == 0]

ECG_TRAIN_DOWNSAMPLED = resample(ECG_TRAIN_NORMAL,
                                      replace=False,
                                      n_samples=len(ECG_TRAIN_ABNORMAL),
                                      random_state=30)

# Concat normal and abnormal
train_set_downsampled = pd.concat([ECG_TRAIN_ABNORMAL, ECG_TRAIN_DOWNSAMPLED])

# Shuffle
train_set_shuffled = train_set_downsampled.sample(frac=1, random_state=30)

# clean outliers from datasets
train_set = clean_outlier(train_set_shuffled)
test_set = clean_outlier(test_set_new)


Scale features of train and test set

In [ ]:
# scaling the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0,1))
# fit the scaler on the train set
scaler.fit(train_set)

# transform both train and test data set with the scaler
train_set_scaled = pd.DataFrame(scaler.transform(train_set))
train_set_scaled.columns=data.columns.values
test_set_scaled = pd.DataFrame(scaler.transform(test_set))
test_set_scaled.columns=data.columns.values

PCA

In [ ]:
x = train_set_scaled.iloc[:, :-1].values
y = train_set_scaled.iloc[:, -1].values

pca = PCA()
X_pca = pca.fit_transform(x)

# initialize variables
cumulative_variance = 0
num_components = 0

# loop over each principal component and add its explained variance to the cumulative variance
while cumulative_variance < 0.8:
    cumulative_variance += pca.explained_variance_ratio_[num_components]
    num_components += 1

# print the number of components required to reach 80% variance
print("Number of components to reach 80% variance:", num_components)

pca = PCA(num_components)
X_pca = pca.fit_transform(x)

# Twee componenten plotten
pca_2components = PCA(n_components=2)
X_pca_2components = pca_2components.fit_transform(x)
df_pca = pd.DataFrame(data=X_pca_2components, columns=['PC1', 'PC2'])
df_pca['Target'] = y
sns.scatterplot(x='PC1', y='PC2', hue='Target', data=df_pca)
plt.show()

In [ ]:
# Colorplot
def colorplot(clf, ax, x, y, h=100, precomputer=None):
    '''
    Overlay the decision areas as colors in an axes.
    
    Input:
        clf: trained classifier
        ax: axis to overlay color mesh on
        x: feature on x-axis
        y: feature on y-axis
        h(optional): steps in the mesh
    '''
    # Create a meshgrid the size of the axis
    xstep = (x.max() - x.min() ) / 20.0
    ystep = (y.max() - y.min() ) / 20.0
    x_min, x_max = x.min() - xstep, x.max() + xstep
    y_min, y_max = y.min() - ystep, y.max() + ystep
    h = max((x_max - x_min, y_max - y_min))/h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    features = np.c_[xx.ravel(), yy.ravel()]
    if precomputer is not None:
        if type(precomputer) is RBFSampler:
            features = precomputer.transform(features)
        elif precomputer is rbf_kernel:
            features = rbf_kernel(features, X)
            
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(features)
    elif hasattr(clf, "predict_proba"):
        Z = clf.predict_proba(features)
    else:
        Z = clf.predict(features)
        
    if len(Z.shape) > 1:
        Z = Z[:, 1]
    
    # Put the result into a color plot
    cm = plt.cm.RdBu_r
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)
    del xx, yy, x_min, x_max, y_min, y_max, Z, cm

KNN

In [ ]:
# KNN classifier
knn = KNeighborsClassifier(n_neighbors = 3, weights='uniform')
knn.fit(X_pca,y)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=30)

# Compute the accuracy of the classifier
accuracy_scores = cross_val_score(knn, X_pca, y, cv=cv, scoring='accuracy')
mean_accuracy_score = accuracy_scores.mean()
print("The mean accuracy score of the classifier is: {:.2f}".format(mean_accuracy_score))

# Compute the precision of the classifier
precision_scores = cross_val_score(knn, X_pca, y, cv=cv, scoring='precision')
mean_precision_score = precision_scores.mean()
print("The mean precision score of the classifier is: {:.2f}".format(mean_precision_score))

# Compute the recall of the classifier
recall_scores = cross_val_score(knn, X_pca, y, cv=cv, scoring='recall')
mean_recall_score = recall_scores.mean()
print("The mean recall score of the classifier is: {:.2f}".format(mean_recall_score))

# Compute the F1 score of the classifier
f1_scores = cross_val_score(knn, X_pca, y, cv=cv, scoring='f1')
mean_f1_score = f1_scores.mean()
print("The mean F1 score of the classifier is: {:.2f}".format(mean_f1_score))

# Compute the weighted F1 score of the classifier
fbeta_scorer = make_scorer(fbeta_score, beta=2)
fbeta_scores = cross_val_score(knn, X_pca, y, cv=cv, scoring=fbeta_scorer)
mean_fbeta_score = fbeta_scores.mean()
print("The mean weighted F1 score of the classifier is: {:.2f}".format(mean_fbeta_score))

# # Finding optimal hyperparameters for recall
# params = {'n_neighbors': [3, 5, 7, 9],
#           'weights': ['uniform', 'distance']}

# grid_search = GridSearchCV(knn, param_grid=params, cv=5, scoring='recall')
# grid_search.fit(X_pca, y)
# print("Best hyperparameters:", grid_search.best_params_)
# print("Best recall score:", grid_search.best_score_)

SVM RBF

In [ ]:
# SVM RBF
svmrbf = SVC(kernel='rbf', gamma=0.1, C=0.1)
svmrbf.fit(X_pca,y)

# Compute the accuracy of the classifier
accuracy_scores = cross_val_score(svmrbf, X_pca, y, cv=cv, scoring='accuracy')
mean_accuracy_score = accuracy_scores.mean()
print("The mean accuracy score of the classifier is: {:.2f}".format(mean_accuracy_score))

# Compute the precision of the classifier
precision_scores = cross_val_score(svmrbf, X_pca, y, cv=cv, scoring='precision')
mean_precision_score = precision_scores.mean()
print("The mean precision score of the classifier is: {:.2f}".format(mean_precision_score))

# Compute the recall of the classifier
recall_scores = cross_val_score(svmrbf, X_pca, y, cv=cv, scoring='recall')
mean_recall_score = recall_scores.mean()
print("The mean recall score of the classifier is: {:.2f}".format(mean_recall_score))

# Compute the F1 score of the classifier
f1_scores = cross_val_score(svmrbf, X_pca, y, cv=cv, scoring='f1')
mean_f1_score = f1_scores.mean()
print("The mean F1 score of the classifier is: {:.2f}".format(mean_f1_score))

# Compute the weighted F1 score of the classifier
fbeta_scorer = make_scorer(fbeta_score, beta=2)
fbeta_scores = cross_val_score(svmrbf, X_pca, y, cv=cv, scoring=fbeta_scorer)
mean_fbeta_score = fbeta_scores.mean()
print("The mean weighted F1 score of the classifier is: {:.2f}".format(mean_fbeta_score))

# # Optimize hyperparameters for highest recall
# param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
# grid_search = GridSearchCV(estimator=svmrbf, param_grid=param_grid, cv=cv, scoring='recall')
# grid_search.fit(X_pca, y)
# print("Best parameters: ", grid_search.best_params_)
# print("Recall score: ", grid_search.best_score_)

SVM polynomial

In [ ]:
# SVM polynomial
svmpoly = SVC(kernel='poly', degree=1, gamma='auto', C=1)
svmpoly.fit(X_pca,y)

# Compute the accuracy of the classifier
accuracy_scores = cross_val_score(svmpoly, X_pca, y, cv=cv, scoring='accuracy')
mean_accuracy_score = accuracy_scores.mean()
print("The mean accuracy score of the classifier is: {:.2f}".format(mean_accuracy_score))

# Compute the precision of the classifier
precision_scores = cross_val_score(svmpoly, X_pca, y, cv=cv, scoring='precision')
mean_precision_score = precision_scores.mean()
print("The mean precision score of the classifier is: {:.2f}".format(mean_precision_score))

# Compute the recall of the classifier
recall_scores = cross_val_score(svmpoly, X_pca, y, cv=cv, scoring='recall')
mean_recall_score = recall_scores.mean()
print("The mean recall score of the classifier is: {:.2f}".format(mean_recall_score))

# Compute the F1 score of the classifier
f1_scores = cross_val_score(svmpoly, X_pca, y, cv=cv, scoring='f1')
mean_f1_score = f1_scores.mean()
print("The mean F1 score of the classifier is: {:.2f}".format(mean_f1_score))

# Compute the weighted F1 score of the classifier
fbeta_scorer = make_scorer(fbeta_score, beta=2)
fbeta_scores = cross_val_score(svmpoly, X_pca, y, cv=cv, scoring=fbeta_scorer)
mean_fbeta_score = fbeta_scores.mean()
print("The mean weighted F1 score of the classifier is: {:.2f}".format(mean_fbeta_score))

# Optimize hyperparameters for highest recall
# param_grid = {'C': [0.1, 1, 10], 'kernel': ['poly'], 'degree': [1, 2, 3, 4], 'gamma': ['scale', 'auto']}
# grid_search = GridSearchCV(svmpoly, param_grid=param_grid, scoring='recall', cv=5)
# grid_search.fit(X_pca, y)
# print("Best hyperparameters:", grid_search.best_params_)
# print("Recall score:", grid_search.best_score_)

Decision tree

In [ ]:
# Decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth= None, min_samples_leaf=16, min_samples_split=2)
decision_tree.fit(X_pca,y)

# Compute the accuracy of the classifier
accuracy_scores = cross_val_score(decision_tree, X_pca, y, cv=cv, scoring='accuracy')
mean_accuracy_score = accuracy_scores.mean()
print("The mean accuracy score of the classifier is: {:.2f}".format(mean_accuracy_score))

# Compute the precision of the classifier
precision_scores = cross_val_score(decision_tree, X_pca, y, cv=cv, scoring='precision')
mean_precision_score = precision_scores.mean()
print("The mean precision score of the classifier is: {:.2f}".format(mean_precision_score))

# Compute the recall of the classifier
recall_scores = cross_val_score(decision_tree, X_pca, y, cv=cv, scoring='recall')
mean_recall_score = recall_scores.mean()
print("The mean recall score of the classifier is: {:.2f}".format(mean_recall_score))

# Compute the F1 score of the classifier
f1_scores = cross_val_score(decision_tree, X_pca, y, cv=cv, scoring='f1')
mean_f1_score = f1_scores.mean()
print("The mean F1 score of the classifier is: {:.2f}".format(mean_f1_score))

# Compute the weighted F1 score of the classifier
fbeta_scorer = make_scorer(fbeta_score, beta=2)
fbeta_scores = cross_val_score(decision_tree, X_pca, y, cv=cv, scoring=fbeta_scorer)
mean_fbeta_score = fbeta_scores.mean()
print("The mean weighted F1 score of the classifier is: {:.2f}".format(mean_fbeta_score))

# # Optimizing hyperparameters for highest recall
# params = {'max_depth': [None, 5, 10, 15, 20],
#           'min_samples_split': [2, 5, 10, 15],
#           'min_samples_leaf': [1, 2, 4, 8, 16]}

# grid_search = GridSearchCV(decision_tree, param_grid=params, cv=5, scoring='recall')
# grid_search.fit(X_pca, y)
# print("Best hyperparameters:", grid_search.best_params_)
# print("Best recall score:", grid_search.best_score_)

Random Forest

In [ ]:
# Random forest classifier
random_forest = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300)
random_forest.fit(X_pca,y)

# Compute the accuracy of the classifier
accuracy_scores = cross_val_score(random_forest, X_pca, y, cv=cv, scoring='accuracy')
mean_accuracy_score = accuracy_scores.mean()
print("The mean accuracy score of the classifier is: {:.2f}".format(mean_accuracy_score))

# Compute the precision of the classifier
precision_scores = cross_val_score(random_forest, X_pca, y, cv=cv, scoring='precision')
mean_precision_score = precision_scores.mean()
print("The mean precision score of the classifier is: {:.2f}".format(mean_precision_score))

# Compute the recall of the classifier
recall_scores = cross_val_score(random_forest, X_pca, y, cv=cv, scoring='recall')
mean_recall_score = recall_scores.mean()
print("The mean recall score of the classifier is: {:.2f}".format(mean_recall_score))

# Compute the F1 score of the classifier
f1_scores = cross_val_score(random_forest, X_pca, y, cv=cv, scoring='f1')
mean_f1_score = f1_scores.mean()
print("The mean F1 score of the classifier is: {:.2f}".format(mean_f1_score))

# Compute the weighted F1 score of the classifier
fbeta_scorer = make_scorer(fbeta_score, beta=2)
fbeta_scores = cross_val_score(random_forest, X_pca, y, cv=cv, scoring=fbeta_scorer)
mean_fbeta_score = fbeta_scores.mean()
print("The mean weighted F1 score of the classifier is: {:.2f}".format(mean_fbeta_score))

# # Finding hyperparameters for highest recall
# params = {'n_estimators': [100, 200, 300],
#           'max_depth': [None, 5, 10],
#           'min_samples_split': [2, 5, 10],
#           'min_samples_leaf': [1, 2, 4]}

# grid_search = GridSearchCV(random_forest, param_grid=params, cv=5, scoring='recall')
# grid_search.fit(X_pca, y)
# print("Best hyperparameters:", grid_search.best_params_)
# print("Best recall score:", grid_search.best_score_)

Linear

In [ ]:
# Linear classifier
linear = LogisticRegression(C=0.1, penalty='l2', max_iter=1000)
linear.fit(X_pca,y)

# Compute the accuracy of the classifier
accuracy_scores = cross_val_score(linear, X_pca, y, cv=cv, scoring='accuracy')
mean_accuracy_score = accuracy_scores.mean()
print("The mean accuracy score of the classifier is: {:.2f}".format(mean_accuracy_score))

# Compute the precision of the classifier
precision_scores = cross_val_score(linear, X_pca, y, cv=cv, scoring='precision')
mean_precision_score = precision_scores.mean()
print("The mean precision score of the classifier is: {:.2f}".format(mean_precision_score))

# Compute the recall of the classifier
recall_scores = cross_val_score(linear, X_pca, y, cv=cv, scoring='recall')
mean_recall_score = recall_scores.mean()
print("The mean recall score of the classifier is: {:.2f}".format(mean_recall_score))

# Compute the F1 score of the classifier
f1_scores = cross_val_score(linear, X_pca, y, cv=cv, scoring='f1')
mean_f1_score = f1_scores.mean()
print("The mean F1 score of the classifier is: {:.2f}".format(mean_f1_score))

# Compute the weighted F1 score of the classifier
fbeta_scorer = make_scorer(fbeta_score, beta=2)
fbeta_scores = cross_val_score(linear, X_pca, y, cv=cv, scoring=fbeta_scorer)
mean_fbeta_score = fbeta_scores.mean()
print("The mean weighted F1 score of the classifier is: {:.2f}".format(mean_fbeta_score))

# # Optimizing hyperparameters for highest recall
# param_grid = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
# grid_search = GridSearchCV(linear, param_grid, cv=5, scoring='recall')
# grid_search.fit(X_pca, y)
# print("Best hyperparameters: ", grid_search.best_params_)
# print("Best recall score: ", grid_search.best_score_)

Combination

In [ ]:
# Combining 3 classifiers with highest recall 
predicted_labels_svmrbf = cross_val_predict(svmrbf, X_pca, y, cv=cv)
predicted_labels_random_forest = cross_val_predict(random_forest, X_pca, y, cv=cv)
predicted_labels_linear = cross_val_predict(linear, X_pca, y, cv=cv)
final_classification = [max(elem) for elem in zip(predicted_labels_svmrbf, predicted_labels_random_forest, predicted_labels_linear)]

y = train_set_scaled.iloc[:, -1].values
correct_labels = y.tolist()

recall = recall_score(correct_labels, final_classification, average='macro')
print(f"The recall score is: {recall:.2f}")

accuracy = accuracy_score(correct_labels, final_classification)
print(f"The accuracy score is: {accuracy:.2f}")

precision = precision_score(correct_labels,final_classification)
print(f"The precision score is: {precision:.2f}")